In [1]:
import pandas as pd
import urllib.parse
from random import shuffle
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [2]:
mt = pd.read_csv("metacritic_data.csv")

In [3]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[:3] + lower_alnum(s)[-3:])
tags = list(zip(mt["id"], mt["Singer"], mt["Album_Name"]))
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages', 'Ranked']
df = pd.DataFrame(columns=cols)

In [ ]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))
# tags = [('divmo','Maya Hawke', 'MOSS')]

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
    except:
      album_infos = []
    
    if not album_infos:
      try:
        # Try search then
        url_tag = urllib.parse.quote(tag)
        album_infos = network.get_album_infos(name=url_tag) # get album by search
      except IndexError as e:
        print("❌ ", tag)
      
    if not album_infos:
      new_tags.append((uid, singer, album))
    else:
      print("✅ ", tag)
      album_infos["Id"] = uid
      df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    # except (IndexError, AttributeError):

  tags = new_tags
  iter += 1

In [7]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages,Ranked
count,9,9,9,9,9,9,9,8,9,9,0,7
unique,9,9,9,9,9,9,7,1,9,9,0,7
top,beak201833,>>>,Beak>,21 September 2018,3.29\n \n/ 5.00.5\n\n ...,"Experimental Rock, Electronic\nKrautrock, Neo-...",,English,"[The Brazilian, Brean Down, Birthday Suit, Har...","[#c0ab6c, #b06546, #989890, #947e4d, #736945]",NaN,#728 for 2018
freq,1,1,1,1,1,1,3,8,1,1,NaN,1


In [8]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
xdf = xdf[['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme','Ranked']]
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked
count,9,9,9,9,9,9,9,8,9,9,7
unique,9,9,9,9,9,9,7,1,9,9,7
top,beak201833,>>>,Beak>,21 September 2018,3.29\n \n/ 5.00.5\n\n ...,"Experimental Rock, Electronic\nKrautrock, Neo-...",,English,"[The Brazilian, Brean Down, Birthday Suit, Har...","[#c0ab6c, #b06546, #989890, #947e4d, #736945]",#728 for 2018
freq,1,1,1,1,1,1,3,8,1,1,1


In [10]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [11]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [12]:
xdf

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked,Num_of_RYM_Ratings
0,beak201833,>>>,Beak>,21 September 2018,3.29,"Experimental Rock, Electronic\nKrautrock, Neo-...","psychedelic, warm, male vocals, mellow, in...",English,"[The Brazilian, Brean Down, Birthday Suit, Har...","[#c0ab6c, #b06546, #989890, #947e4d, #736945]",#728 for 2018,839
0,gilscottheron2020werven,Re-Ron,Gil Scott-Heron,1984,3.50,Electro\nSpoken Word,,NaN,"[Re-Ron (Long Version), Re-Ron (Special Mix)]","[#7b7b7b, #c1c1c1, #c8c9bb, #93a5af, #9c9c9c]",#899 for 1984,31
0,danielblumberg2020ononon,On&On,Daniel Blumberg,31 July 2020,3.30,"Indie Rock, Art Rock\nSinger-Songwriter, Exper...","male vocals, repetitive, passionate, noctur...",English,"[On & On, Sidestep Summer, On&On&On, Bound, Si...","[#7d8182, #bcbcb8, #7a7c7a, #8cbbd9]","#1,126 for 2020",230
0,metallica2020sm2ive,S&M,Metallica,23 November 1999,3.81,"Thrash Metal, Symphonic Metal, Heavy Metal\nHa...",,English,"[The Ecstasy of Gold, The Call of Ktulu, Maste...","[#7a7a7a, #7a8191, #9d989e, #aaa1b6]","#219 overall, #126 for live",481
0,lukasnelsonpromiseofthereal2021afeart,A Few Stars Apart,Lukas Nelson & Promise of the Real,11 June 2021,3.34,Country Rock,,English,"[We'll Be Alright, Perennial Bloom (Back to Yo...","[#859db9, #2e4358, #c5c7c6, #a0a09e]",NaN,62
0,edsheeran2021equals,= [Equals],Ed Sheeran,29 October 2021,1.78,"Pop\nSynthpop, Folk Pop, Dance-Pop, Pop Rock","male vocals, romantic, melodic, sentimental...",English,"[Tides, Shivers, First Times, Bad Habits, Over...","[#ce0b2d, #7f081b, #8c7a7a, #bd2e40, #9b8789]",NaN,1497
0,andyouwillknowusthetrailofdead2022xibnow,XI: Bleed Here Now,...And You Will Know Us by the Trail of Dead,30 September 2022,3.24,"Alternative Rock\nProgressive Rock, Art Rock, ...","male vocals, melodic, epic",English,"[Our Epic Attempts, Long Distance Hell, Field ...","[#17265f, #203874, #384c87, #7a7a7a]",#709 for 2022,265
0,sonicyouth2022inotin,In / Out / In,Sonic Youth,18 March 2022,3.33,"Noise Rock, Experimental Rock\nFree Improvisat...","noisy, atmospheric, instrumental, improvisa...",English,"[Basement Contender, In & Out, Machine, Social...","[#969b94, #ffe2a9, #47663a, #aa9b74, #696c4f]","#9 for 2022, #2,197 overall",411
0,stpaulthebrokenbones2022theast,The Alien Coast,St. Paul & The Broken Bones,28 January 2022,3.08,"Rhythm & Blues, Soul, Psychedelic Soul\nProgre...","psychedelic, progressive",English,"[3000 AD Mass, Bermejo and the Devil, Minotaur...","[#baada7, #9182c3, #5d8771, #c6c6b3, #1f4569, ...","#1,604 for 2022",151


In [13]:
xdf.to_csv("rym_add.csv")